In [1]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [2]:
import numpy as np

from constant import SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import resolve, Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from io import BytesIO
from IPython.display import Audio, display
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, ImageOps

In [3]:
def create_spectrogram(signal, settings):
    spectrogram = Spectrogram()
    strategy = Mel(signal, settings)
    spectrogram.strategy = strategy

    return spectrogram.generate()

In [5]:
dataset = Dataset('signal')
dataframe = dataset.load()

In [6]:
path = SETTINGS.joinpath('spectrogram.json')
ss = Settings.from_file(path)

In [7]:
path = SETTINGS.joinpath('dereverberate.json')
ds = Settings.from_file(path)

In [8]:
signal = deepcopy(
    dataframe.at[0, 'signal']
)

signal.filter(
    ss.butter_lowcut,
    ss.butter_highcut
)

signal.normalize()

signal.dereverberate(ds)

spectrogram = np.frompyfunc(
    lambda x, y: create_spectrogram(x, y),
    2,
    1
)(signal, ss)

image = Image.fromarray(~spectrogram)
image.save('test.png', format='png')
# image.show()
# display(image)

# audio = Audio(
#     data=signal.data,
#     rate=signal.rate,
#     autoplay=False
# )

# display(audio)